In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string
import sklearn

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = '/home/manikya_varshney/Documents/Python/Yale/h01-20200818-10files/final(Cleaned and Analysed)_h01-20200818-153021.csv'
data = pd.read_csv(path, low_memory=False)

data_Imp = data.copy()
data_Ada = data.copy()
data_Ne = data.copy()
data_Sd = data.copy()
data_Purp = data.copy()
data_Pe = data.copy()

# 1_Implementation

In [3]:
keywords_Imp = ['stay at home' , 'do your part', 'Responsible', 
            'home', 'house', 'cancel', 'shutdown', 'postpone',
            'school closure', 'Closure', 'business closure',
            'suspension', 'quarantine', 'lockdown', 'social distance', 
            'social distancing', 'self quarantine', 'isolat', '6-feet',
            'distance', '#clubquarantine', '#quarantinelife', '#quarantineacitivites']

In [4]:
#Convert to lower
for i in range(len(keywords_Imp)): 
    keywords_Imp[i] = keywords_Imp[i].lower()

#Remove punctuations   
for i in range(len(keywords_Imp)): 
    keywords_Imp[i] = keywords_Imp[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Imp)): 
    keywords_Imp[i] = keywords_Imp[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Imp_tokens = [sub.split() for sub in keywords_Imp] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Imp_filtered=remove_stopwords(keywords_Imp)

#Stemming
ps = PorterStemmer()
keywords_Imp_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Imp_filtered]
keywords_Imp_stem = [" ".join(sentence) for sentence in keywords_Imp_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Imp_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Imp_filtered]
keywords_Imp_final = [" ".join(sentence) for sentence in keywords_Imp_lem]

In [5]:
data_Imp

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   
7745  1.295766e+18  1.027784e+08         NaN                   Bahamas   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   
7745                            https://t.co/K25QaQYTtz   

                                                  final  
0              wear clone helmet instead mask to school  
1     sc senate to reconvene sept to discus covid19 ...  
2     the covid19 crisis pres trump always honest se...  
3     gon na act like  ahead the curve sell face mas...  
4     the covid19 pandemic create unprecedented barr...  
...                                                 ...  
7741  everyone see republican man sends methey feel ...  
7742  meanwhile canadian constantly told must sacrif...  
7743  join u tuesday important backtoschool chat the...  
7744  due to the laughable recount death the uk the ...  
7745                                                NaN  

[7746 rows x 6 columns]

In [6]:
 def fuzzy_Imp(row):
        keyword_match_Imp, score_Imp = process.extractOne(row['final'], keywords_Imp_final, scorer = fuzz.partial_ratio)
        row['final_score_Imp'] = score_Imp
        row['final_keyword_match_Imp'] = keyword_match_Imp
        return row

In [7]:
nan_value = float("NaN")
data_Imp['final'].replace("", nan_value, inplace=True)

In [8]:
data_Imp.dropna(subset = ["final"], inplace=True)

In [9]:
data_Imp

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7740  1.295710e+18  1.213801e+09         NaN                     MD/DC   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7740  NEW: Is your state reporting comprehensive dat...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   

                                                  final  
0              wear clone helmet instead mask to school  
1     sc senate to reconvene sept to discus covid19 ...  
2     the covid19 crisis pres trump always honest se...  
3     gon na act like  ahead the curve sell face mas...  
4     the covid19 pandemic create unprecedented barr...  
...                                                 ...  
7740  new your state reporting comprehensive data co...  
7741  everyone see republican man sends methey feel ...  
7742  meanwhile canadian constantly told must sacrif...  
7743  join u tuesday important backtoschool chat the...  
7744  due to the laughable recount death the uk the ...  

[7730 rows x 6 columns]

In [10]:
interim_Imp = data_Imp.apply(fuzzy_Imp, axis=1)

In [11]:
interim_Imp

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7740  1.295710e+18  1.213801e+09         NaN                     MD/DC   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7740  NEW: Is your state reporting comprehensive dat...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   

                                                  final  final_score_Imp  \
0              wear clone helmet instead mask to school               60   
1     sc senate to reconvene sept to discus covid19 ...               60   
2     the covid19 crisis pres trump always honest se...               75   
3     gon na act like  ahead the curve sell face mas...               57   
4     the covid19 pandemic create unprecedented barr...               58   
...                                                 ...              ...   
7740  new your state reporting comprehensive data co...               67   
7741  everyone see republican man sends methey feel ...               60   
7742  meanwhile canadian constantly told must sacrif...               62   
7743  join u tuesday important backtoschool chat the...               57   
7744  due to the laughable recount death the uk the ...              100   

     final_keyword_match_Imp  
0                      6feet  
1                      6feet  
2                       home  
3                    closure  
4               stay at home  
...                      ...  
7740            do your part  
7741                   house  
7742                distance  
7743          school closure  
7744              quarantine  

[7730 rows x 8 columns]

In [12]:
denominator_Imp = interim_Imp.shape[0]

In [13]:
interim_Imp['final_score_Imp'] = interim_Imp['final_score_Imp'].astype(int)

In [14]:
interim_Imp = interim_Imp[interim_Imp['final_score_Imp'] == 100]

In [15]:
interim_Imp

id       user_id coordinates                    location  \
10    1.295805e+18  2.232332e+07         NaN  Nairobi/Kyiv/Nassau/Kosovo   
14    1.295805e+18  2.492172e+08         NaN                         NaN   
15    1.295805e+18  1.466334e+07         NaN                    New York   
16    1.295805e+18  7.136334e+07         NaN                         NaN   
17    1.295805e+18  1.210700e+18         NaN                         NaN   
...            ...           ...         ...                         ...   
7728  1.295775e+18  5.077292e+07         NaN                   Nashville   
7730  1.295737e+18  7.623898e+17         NaN                         NaN   
7732  1.295765e+18  1.713118e+07         NaN             Little Rock, AR   
7738  1.294132e+18  3.810085e+08         NaN       🏡Bikaner, Rajasthan,    
7744  1.295096e+18  2.666017e+09         NaN                  Nottingham   

                     extended_tweet_full_text_duplicate  \
10    Hey, friends. . . I know a lot of y'all #homes...   
14    #TodayInLatinAmerica: Thousands of Argentines ...   
15    Antonio Argibay, AIA, managing principal of Me...   
16    #VIU prof. Dr. Aggie Weighill needs your input...   
17    Why the Lockdown Should Last Longer https://t....   
...                                                 ...   
7728  @darrenrovell It’s one Titans game in Sept., K...   
7730  @realDonaldTrump Cuomo stuffed Thousands of Co...   
7732  Nursing home COVID-19 reporting grows less tra...   
7738  आयुर्वेद के भविष्य को ऐसे  दाव पर मत लगाओ यह क...   
7744  Due to the laughable "recount" of deaths in th...   

                                                  final  final_score_Imp  \
10    hey friend know lot yall homeschool supportive...              100   
14    todayinlatinamerica thousand argentine protest...              100   
15    antonio argibay aia manage principal meridian ...              100   
16    viu prof dr aggie weighill need your input can...              100   
17               the lockdown last longer via hilarious              100   
...                                                 ...              ...   
7728   one titan game sept karen  ask majority ameri...              100   
7730  cuomo stuffed thousand covid patient nursing h...              100   
7732  nursing home covid19 reporting grows less tran...              100   
7738  आयुर्वेद के भविष्य को ऐसे दाव पर मत लगाओ यह को...              100   
7744  due to the laughable recount death the uk the ...              100   

     final_keyword_match_Imp  
10                      home  
14                  lockdown  
15                      home  
16                  distance  
17                  lockdown  
...                      ...  
7728                   house  
7730                    home  
7732                    home  
7738                postpone  
7744              quarantine  

[1347 rows x 8 columns]

In [16]:
numerator_Imp = interim_Imp.shape[0]

In [17]:
proportion_Imp = (numerator_Imp/denominator_Imp)
print(proportion_Imp)

0.17425614489003882


# 2_Adaptation

In [18]:
keywords_Ada = ['school from home' , 'learn', 'remote', 'school food service', 
            'online shopping', 'online purchase', 'online church', 'delivery',
            'drive thru', 'to go', 'take out', 'Tiktok', 'Netflix', 'telework', 
            'zoom', 'telehealth', 'telemedicine', 'work from home', 'wfh',
            'working at home', 'working remotely', 'online meeting']

In [19]:
#Convert to lower
for i in range(len(keywords_Ada)): 
    keywords_Ada[i] = keywords_Ada[i].lower()

#Remove punctuations   
for i in range(len(keywords_Ada)): 
    keywords_Ada[i] = keywords_Ada[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Ada)): 
    keywords_Ada[i] = keywords_Ada[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Ada_tokens = [sub.split() for sub in keywords_Ada] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Ada_filtered=remove_stopwords(keywords_Ada)

#Stemming
ps = PorterStemmer()
keywords_Ada_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Ada_filtered]
keywords_Ada_stem = [" ".join(sentence) for sentence in keywords_Ada_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Ada_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Ada_filtered]
keywords_Ada_final = [" ".join(sentence) for sentence in keywords_Ada_lem]

In [20]:
data_Ada

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   
7745  1.295766e+18  1.027784e+08         NaN                   Bahamas   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   
7745                            https://t.co/K25QaQYTtz   

                                                  final  
0              wear clone helmet instead mask to school  
1     sc senate to reconvene sept to discus covid19 ...  
2     the covid19 crisis pres trump always honest se...  
3     gon na act like  ahead the curve sell face mas...  
4     the covid19 pandemic create unprecedented barr...  
...                                                 ...  
7741  everyone see republican man sends methey feel ...  
7742  meanwhile canadian constantly told must sacrif...  
7743  join u tuesday important backtoschool chat the...  
7744  due to the laughable recount death the uk the ...  
7745                                                NaN  

[7746 rows x 6 columns]

In [21]:
 def fuzzy_Ada(row):
        keyword_match_Ada, score_Ada = process.extractOne(row['final'], keywords_Ada_final, scorer = fuzz.partial_ratio)
        row['final_score_Ada'] = score_Ada
        row['final_keyword_match_Ada'] = keyword_match_Ada
        return row

In [22]:
nan_value = float("NaN")
data_Ada['final'].replace("", nan_value, inplace=True)

In [23]:
data_Ada.dropna(subset = ["final"], inplace=True)

In [24]:
data_Ada

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7740  1.295710e+18  1.213801e+09         NaN                     MD/DC   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7740  NEW: Is your state reporting comprehensive dat...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   

                                                  final  
0              wear clone helmet instead mask to school  
1     sc senate to reconvene sept to discus covid19 ...  
2     the covid19 crisis pres trump always honest se...  
3     gon na act like  ahead the curve sell face mas...  
4     the covid19 pandemic create unprecedented barr...  
...                                                 ...  
7740  new your state reporting comprehensive data co...  
7741  everyone see republican man sends methey feel ...  
7742  meanwhile canadian constantly told must sacrif...  
7743  join u tuesday important backtoschool chat the...  
7744  due to the laughable recount death the uk the ...  

[7730 rows x 6 columns]

In [25]:
interim_Ada = data_Ada.apply(fuzzy_Ada, axis=1)

In [26]:
denominator_Ada = interim_Ada.shape[0]

In [27]:
interim_Ada['final_score_Ada'] = interim_Ada['final_score_Ada'].astype(int)

In [28]:
interim_Ada = interim_Ada[interim_Ada['final_score_Ada'] == 100]

In [29]:
numerator_Ada = interim_Ada.shape[0]

In [30]:
proportion_Ada = (numerator_Ada/denominator_Ada)
print(proportion_Ada)

0.035834411384217335


# 3_Negative Emotion

In [31]:
keywords_Ne = ['bored' , 'lonely', 'stress', 
            'anxiety', 'scared', 'worry', 'end', 'cabin fever',
            '#sideeffectsofquarantinelife', 'tissue paper', 'toilet paper']

In [32]:
#Convert to lower
for i in range(len(keywords_Ne)): 
    keywords_Ne[i] = keywords_Ne[i].lower()

#Remove punctuations   
for i in range(len(keywords_Ne)): 
    keywords_Ne[i] = keywords_Ne[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Ne)): 
    keywords_Ne[i] = keywords_Ne[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Ne_tokens = [sub.split() for sub in keywords_Ne] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Ne_filtered=remove_stopwords(keywords_Ne)

#Stemming
ps = PorterStemmer()
keywords_Ne_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Ne_filtered]
keywords_Ne_stem = [" ".join(sentence) for sentence in keywords_Ne_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Ne_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Ne_filtered]
keywords_Ne_final = [" ".join(sentence) for sentence in keywords_Ne_lem]

In [33]:
data_Ne

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   
7745  1.295766e+18  1.027784e+08         NaN                   Bahamas   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   
7745                            https://t.co/K25QaQYTtz   

                                                  final  
0              wear clone helmet instead mask to school  
1     sc senate to reconvene sept to discus covid19 ...  
2     the covid19 crisis pres trump always honest se...  
3     gon na act like  ahead the curve sell face mas...  
4     the covid19 pandemic create unprecedented barr...  
...                                                 ...  
7741  everyone see republican man sends methey feel ...  
7742  meanwhile canadian constantly told must sacrif...  
7743  join u tuesday important backtoschool chat the...  
7744  due to the laughable recount death the uk the ...  
7745                                                NaN  

[7746 rows x 6 columns]

In [34]:
 def fuzzy_Ne(row):
        keyword_match_Ne, score_Ne = process.extractOne(row['final'], keywords_Ne_final, scorer = fuzz.partial_ratio)
        row['final_score_Ne'] = score_Ne
        row['final_keyword_match_Ne'] = keyword_match_Ne
        return row

In [35]:
nan_value = float("NaN")
data_Ne['final'].replace("", nan_value, inplace=True)

In [36]:
data_Ne.dropna(subset = ["final"], inplace=True)

In [37]:
data_Ne

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7740  1.295710e+18  1.213801e+09         NaN                     MD/DC   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7740  NEW: Is your state reporting comprehensive dat...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   

                                                  final  
0              wear clone helmet instead mask to school  
1     sc senate to reconvene sept to discus covid19 ...  
2     the covid19 crisis pres trump always honest se...  
3     gon na act like  ahead the curve sell face mas...  
4     the covid19 pandemic create unprecedented barr...  
...                                                 ...  
7740  new your state reporting comprehensive data co...  
7741  everyone see republican man sends methey feel ...  
7742  meanwhile canadian constantly told must sacrif...  
7743  join u tuesday important backtoschool chat the...  
7744  due to the laughable recount death the uk the ...  

[7730 rows x 6 columns]

In [38]:
interim_Ne = data_Ne.apply(fuzzy_Ne, axis=1)

In [39]:
denominator_Ne = interim_Ne.shape[0]

In [40]:
interim_Ne['final_score_Ne'] = interim_Ne['final_score_Ne'].astype(int)

In [41]:
interim_Ne = interim_Ne[interim_Ne['final_score_Ne'] == 100]

In [42]:
numerator_Ne = interim_Ne.shape[0]

In [43]:
proportion_Ne = (numerator_Ne/denominator_Ne)
print(proportion_Ne)

0.05058214747736093


# 4_Social Disruption

In [44]:
keywords_Sd = ['social functions' , 'gathering', 'empty streets', 
            'interaction', 'large', 'no cars', 'non-essential',
            'travel', 'unnecessary', 'crowd',]

In [45]:
#Convert to lower
for i in range(len(keywords_Sd)): 
    keywords_Sd[i] = keywords_Sd[i].lower()

#Remove punctuations   
for i in range(len(keywords_Sd)): 
    keywords_Sd[i] = keywords_Sd[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Sd)): 
    keywords_Sd[i] = keywords_Sd[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Sd_tokens = [sub.split() for sub in keywords_Sd] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Sd_filtered=remove_stopwords(keywords_Sd)

#Stemming
ps = PorterStemmer()
keywords_Sd_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Sd_filtered]
keywords_Sd_stem = [" ".join(sentence) for sentence in keywords_Sd_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Sd_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Sd_filtered]
keywords_Sd_final = [" ".join(sentence) for sentence in keywords_Sd_lem]

In [46]:
data_Sd

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   
7745  1.295766e+18  1.027784e+08         NaN                   Bahamas   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   
7745                            https://t.co/K25QaQYTtz   

                                                  final  
0              wear clone helmet instead mask to school  
1     sc senate to reconvene sept to discus covid19 ...  
2     the covid19 crisis pres trump always honest se...  
3     gon na act like  ahead the curve sell face mas...  
4     the covid19 pandemic create unprecedented barr...  
...                                                 ...  
7741  everyone see republican man sends methey feel ...  
7742  meanwhile canadian constantly told must sacrif...  
7743  join u tuesday important backtoschool chat the...  
7744  due to the laughable recount death the uk the ...  
7745                                                NaN  

[7746 rows x 6 columns]

In [47]:
 def fuzzy_Sd(row):
        keyword_match_Sd, score_Sd = process.extractOne(row['final'], keywords_Sd_final, scorer = fuzz.partial_ratio)
        row['final_score_Sd'] = score_Sd
        row['final_keyword_match_Sd'] = keyword_match_Sd
        return row

In [48]:
nan_value = float("NaN")
data_Sd['final'].replace("", nan_value, inplace=True)

In [49]:
data_Sd.dropna(subset = ["final"], inplace=True)

In [50]:
data_Sd

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7740  1.295710e+18  1.213801e+09         NaN                     MD/DC   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7740  NEW: Is your state reporting comprehensive dat...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   

                                                  final  
0              wear clone helmet instead mask to school  
1     sc senate to reconvene sept to discus covid19 ...  
2     the covid19 crisis pres trump always honest se...  
3     gon na act like  ahead the curve sell face mas...  
4     the covid19 pandemic create unprecedented barr...  
...                                                 ...  
7740  new your state reporting comprehensive data co...  
7741  everyone see republican man sends methey feel ...  
7742  meanwhile canadian constantly told must sacrif...  
7743  join u tuesday important backtoschool chat the...  
7744  due to the laughable recount death the uk the ...  

[7730 rows x 6 columns]

In [51]:
interim_Sd = data_Sd.apply(fuzzy_Sd, axis=1)

In [52]:
denominator_Sd = interim_Sd.shape[0]

In [53]:
interim_Sd['final_score_Sd'] = interim_Sd['final_score_Sd'].astype(int)

In [54]:
interim_Sd = interim_Sd[interim_Sd['final_score_Sd'] == 100]

In [55]:
numerator_Sd = interim_Sd.shape[0]

In [56]:
proportion_Sd = (numerator_Sd/denominator_Sd)
print(proportion_Sd)

0.02147477360931436


# 5_Purpose

In [57]:
keywords_Purp = ['Flatten the curve' , 'Slow the spread', 'slow transmission', 
            'protect', 'save', '#stayhomesavelives']

In [58]:
#Convert to lower
for i in range(len(keywords_Purp)): 
    keywords_Purp[i] = keywords_Purp[i].lower()

#Remove punctuations   
for i in range(len(keywords_Purp)): 
    keywords_Purp[i] = keywords_Purp[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Purp)): 
    keywords_Purp[i] = keywords_Purp[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Purp_tokens = [sub.split() for sub in keywords_Purp] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Purp_filtered=remove_stopwords(keywords_Purp)

#Stemming
ps = PorterStemmer()
keywords_Purp_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Purp_filtered]
keywords_Purp_stem = [" ".join(sentence) for sentence in keywords_Purp_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Purp_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Purp_filtered]
keywords_Purp_final = [" ".join(sentence) for sentence in keywords_Purp_lem]

In [59]:
data_Purp

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   
7745  1.295766e+18  1.027784e+08         NaN                   Bahamas   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   
7745                            https://t.co/K25QaQYTtz   

                                                  final  
0              wear clone helmet instead mask to school  
1     sc senate to reconvene sept to discus covid19 ...  
2     the covid19 crisis pres trump always honest se...  
3     gon na act like  ahead the curve sell face mas...  
4     the covid19 pandemic create unprecedented barr...  
...                                                 ...  
7741  everyone see republican man sends methey feel ...  
7742  meanwhile canadian constantly told must sacrif...  
7743  join u tuesday important backtoschool chat the...  
7744  due to the laughable recount death the uk the ...  
7745                                                NaN  

[7746 rows x 6 columns]

In [60]:
 def fuzzy_Purp(row):
        keyword_match_Purp, score_Purp = process.extractOne(row['final'], keywords_Purp_final, scorer = fuzz.partial_ratio)
        row['final_score_Purp'] = score_Purp
        row['final_keyword_match_Purp'] = keyword_match_Purp
        return row

In [61]:
nan_value = float("NaN")
data_Purp['final'].replace("", nan_value, inplace=True)

In [62]:
data_Purp.dropna(subset = ["final"], inplace=True)

In [63]:
data_Purp

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7740  1.295710e+18  1.213801e+09         NaN                     MD/DC   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7740  NEW: Is your state reporting comprehensive dat...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   

                                                  final  
0              wear clone helmet instead mask to school  
1     sc senate to reconvene sept to discus covid19 ...  
2     the covid19 crisis pres trump always honest se...  
3     gon na act like  ahead the curve sell face mas...  
4     the covid19 pandemic create unprecedented barr...  
...                                                 ...  
7740  new your state reporting comprehensive data co...  
7741  everyone see republican man sends methey feel ...  
7742  meanwhile canadian constantly told must sacrif...  
7743  join u tuesday important backtoschool chat the...  
7744  due to the laughable recount death the uk the ...  

[7730 rows x 6 columns]

In [64]:
interim_Purp = data_Purp.apply(fuzzy_Purp, axis=1)

In [65]:
denominator_Purp = interim_Purp.shape[0]

In [66]:
interim_Purp['final_score_Purp'] = interim_Purp['final_score_Purp'].astype(int)

In [67]:
interim_Purp = interim_Purp[interim_Purp['final_score_Purp'] == 100]

In [68]:
numerator_Purp = interim_Purp.shape[0]

In [69]:
proportion_Purp = (numerator_Purp/denominator_Purp)
print(proportion_Purp)

0.022380336351875807


# 6_Positive Emotion

In [70]:
keywords_Pe = ['silver lining' , 'optimistic', 'hope', 
            'bright side', 'Safe', '#togetherapart']

In [71]:
#Convert to lower
for i in range(len(keywords_Pe)): 
    keywords_Pe[i] = keywords_Pe[i].lower()

#Remove punctuations   
for i in range(len(keywords_Pe)): 
    keywords_Pe[i] = keywords_Pe[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Pe)): 
    keywords_Pe[i] = keywords_Pe[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Pe_tokens = [sub.split() for sub in keywords_Pe] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Pe_filtered=remove_stopwords(keywords_Pe)

#Stemming
ps = PorterStemmer()
keywords_Pe_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Pe_filtered]
keywords_Pe_stem = [" ".join(sentence) for sentence in keywords_Pe_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Pe_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Pe_filtered]
keywords_Pe_final = [" ".join(sentence) for sentence in keywords_Pe_lem]

In [72]:
data_Pe

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   
7745  1.295766e+18  1.027784e+08         NaN                   Bahamas   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   
7745                            https://t.co/K25QaQYTtz   

                                                  final  
0              wear clone helmet instead mask to school  
1     sc senate to reconvene sept to discus covid19 ...  
2     the covid19 crisis pres trump always honest se...  
3     gon na act like  ahead the curve sell face mas...  
4     the covid19 pandemic create unprecedented barr...  
...                                                 ...  
7741  everyone see republican man sends methey feel ...  
7742  meanwhile canadian constantly told must sacrif...  
7743  join u tuesday important backtoschool chat the...  
7744  due to the laughable recount death the uk the ...  
7745                                                NaN  

[7746 rows x 6 columns]

In [73]:
 def fuzzy_Pe(row):
        keyword_match_Pe, score_Pe = process.extractOne(row['final'], keywords_Pe_final, scorer = fuzz.partial_ratio)
        row['final_score_Pe'] = score_Pe
        row['final_keyword_match_Pe'] = keyword_match_Pe
        return row

In [74]:
nan_value = float("NaN")
data_Pe['final'].replace("", nan_value, inplace=True)

In [75]:
data_Pe.dropna(subset = ["final"], inplace=True)

In [76]:
data_Pe

id       user_id coordinates                  location  \
0     1.295805e+18  1.278716e+18         NaN                       NaN   
1     1.295805e+18  9.191072e+08         NaN          Myrtle Beach, SC   
2     1.295805e+18  4.378954e+08         NaN             Roseville, CA   
3     1.295805e+18  2.580088e+09         NaN                       NaN   
4     1.295805e+18  8.560174e+07         NaN          Honolulu, Hawaii   
...            ...           ...         ...                       ...   
7740  1.295710e+18  1.213801e+09         NaN                     MD/DC   
7741  1.295803e+18  7.455473e+17         NaN  The Bay Area, California   
7742  1.295797e+18  1.291778e+09         NaN                       NaN   
7743  1.295004e+18  3.486331e+08         NaN                   Ontario   
7744  1.295096e+18  2.666017e+09         NaN                Nottingham   

                     extended_tweet_full_text_duplicate  \
0     can we wear clone helmets instead of masks to ...   
1     SC Senate to reconvene in Sept. to discuss COV...   
2     @JohnAvlon During the COVID-19 crisis, Pres. T...   
3     We just gonna act like @JamesStormBrand wasn’t...   
4     The COVID-19 pandemic has created unprecedente...   
...                                                 ...   
7740  NEW: Is your state reporting comprehensive dat...   
7741  Just so everyone can see what a republican man...   
7742  Meanwhile, Canadians are constantly told they ...   
7743  Join us Tuesday for an important back-to-schoo...   
7744  Due to the laughable "recount" of deaths in th...   

                                                  final  
0              wear clone helmet instead mask to school  
1     sc senate to reconvene sept to discus covid19 ...  
2     the covid19 crisis pres trump always honest se...  
3     gon na act like  ahead the curve sell face mas...  
4     the covid19 pandemic create unprecedented barr...  
...                                                 ...  
7740  new your state reporting comprehensive data co...  
7741  everyone see republican man sends methey feel ...  
7742  meanwhile canadian constantly told must sacrif...  
7743  join u tuesday important backtoschool chat the...  
7744  due to the laughable recount death the uk the ...  

[7730 rows x 6 columns]

In [77]:
interim_Pe = data_Pe.apply(fuzzy_Pe, axis=1)

In [78]:
denominator_Pe = interim_Pe.shape[0]

In [79]:
interim_Pe['final_score_Pe'] = interim_Pe['final_score_Pe'].astype(int)

In [80]:
interim_Pe = interim_Pe[interim_Pe['final_score_Pe'] == 100]

In [81]:
numerator_Pe = interim_Pe.shape[0]

In [82]:
proportion_Pe = (numerator_Pe/denominator_Pe)
print(proportion_Pe)

0.031177231565329883


# All Proportions

In [83]:
print(proportion_Imp)

0.17425614489003882


In [84]:
print(proportion_Ada)

0.035834411384217335


In [85]:
print(proportion_Ne)

0.05058214747736093


In [86]:
print(proportion_Sd)

0.02147477360931436


In [87]:
print(proportion_Purp)

0.022380336351875807


In [88]:
print(proportion_Pe)

0.031177231565329883


# Checking for same Tweet in different Facet (15 Cases)

### 1_Implementation and 2_Adaptation

In [90]:
pd.merge(interim_Imp, interim_Ada, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x             location_x  \
0    1.293136e+18  2.231783e+09           NaN         Washington, DC   
1    1.293618e+18  2.218081e+07           NaN             Boston, MA   
2    1.293930e+18  4.007672e+07           NaN                    NaN   
3    1.294287e+18  9.143157e+17           NaN          Mumbai, India   
4    1.294289e+18  2.520480e+07           NaN                    NaN   
..            ...           ...           ...                    ...   
100  1.295806e+18  1.562519e+09           NaN  HQ: San Francisco, CA   
101  1.295806e+18  1.029132e+18           NaN                    NaN   
102  1.295806e+18  8.228218e+17           NaN        Wholesomeville😇   
103  1.295806e+18  2.117253e+08           NaN        Florence, Y'ALL   
104  1.295806e+18  1.772784e+08           NaN                    NaN   

                  extended_tweet_full_text_duplicate_x  \
0    Over 36M Nigerian children were affected by #C...   
1    Even before #COVID, only 2 out of 10 employees...   
2    We got remote access into @LittleMix's quarant...   
3    Many need work from home in this post lockdown...   
4    To all the Parents working from home and also ...   
..                                                 ...   
100  Want to create a more engaging and productive ...   
101  @MonaMelanated Beginning of quarantine I learn...   
102  @TwerkingThug @MCoveteur It don’t. Even workin...   
103  @iambiggc Working from home it is SO easy to g...   
104  I thought I’d like working from home for these...   

                                               final_x  final_score_Imp  \
0    36m nigerian child affected covid19 school clo...              100   
1    even covid 2 out 10 employee say close friend ...              100   
2    get remote access quarantine give u the lowdow...              100   
3    many need work from home post lockdown pls sta...              100   
4    to the parent work from home also deal kid dis...              100   
..                                                 ...              ...   
100  want to create engage productive work environm...              100   
101  begin quarantine learn to cook many dish the f...              100   
102   even work from home front the pc screen your ...              100   
103                work from home easy to get distract              100   
104  thought  like work from home next two week  da...              100   

    final_keyword_match_Imp     user_id_y coordinates_y  \
0            school closure  2.231783e+09           NaN   
1                    isolat  2.218081e+07           NaN   
2                quarantine  4.007672e+07           NaN   
3                      home  9.143157e+17           NaN   
4                      home  2.520480e+07           NaN   
..                      ...           ...           ...   
100                    home  1.562519e+09           NaN   
101              quarantine  1.029132e+18           NaN   
102                    home  8.228218e+17           NaN   
103                    home  2.117253e+08           NaN   
104                    home  1.772784e+08           NaN   

                location_y               extended_tweet_full_text_duplicate_y  \
0           Washington, DC  Over 36M Nigerian children were affected by #C...   
1               Boston, MA  Even before #COVID, only 2 out of 10 employees...   
2                      NaN  We got remote access into @LittleMix's quarant...   
3            Mumbai, India  Many need work from home in this post lockdown...   
4                      NaN  To all the Parents working from home and also ...   
..                     ...                                                ...   
100  HQ: San Francisco, CA  Want to create a more engaging and productive ...   
101                    NaN  @MonaMelanated Beginning of quarantine I learn...   
102        Wholesomeville😇  @TwerkingThug @MCoveteur It don’t. Even workin...   
103        Florence, Y'ALL  @iamb

### 1_Implementation and 3_Negative Emotion

In [91]:
pd.merge(interim_Imp, interim_Ne, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x          location_x  \
0   1.272693e+18  9.058951e+17           NaN                 NaN   
1   1.279987e+18  3.391316e+07           NaN       New York, USA   
2   1.288762e+18  2.354005e+08           NaN          London, UK   
3   1.292240e+18  1.187517e+18           NaN           Vancouver   
4   1.293899e+18  2.443667e+09           NaN          Boston, MA   
..           ...           ...           ...                 ...   
70  1.295806e+18  1.286625e+18           NaN                 NaN   
71  1.295806e+18  1.923847e+07           NaN                  UK   
72  1.295806e+18  1.233970e+18           NaN                 NaN   
73  1.295806e+18  1.177414e+18           NaN                 NaN   
74  1.295806e+18  2.364339e+08           NaN  Clitheroe, England   

                 extended_tweet_full_text_duplicate_x  \
0   She isn’t in need, and she surely isn’t sugges...   
1   Mom surprises son with best friend reunion aft...   
2   12 weeks ago we had an idea to make a movie du...   
3   Don’t forget to practice social distancing y’a...   
4   Waiting for corona to end like...\n\n#jennifer...   
..                                                ...   
70  Please sign to end this lockdown  https://t.co...   
71  You know these adverts about people putting lo...   
72  @dg_grrl If getting help to mom depends on the...   
73  You wanna see social distancing?\nLend a mf so...   
74  @mindykaling New quarantine friend https://t.c...   

                                              final_x  final_score_Imp  \
0    need surely  suggest anything quarantine get ...              100   
1   mom surprise son best friend reunion month qua...              100   
2   12 week ago idea to make movie lockdown host t...              100   
3    forget to practice social distance  all‼ migh...              100   
4   wait corona to end like jennifergrout muslimar...              100   
..                                                ...              ...   
70                        please sign to end lockdown              100   
71  know advert people put lockdown weight well u ...              100   
72  get help to mom depends the ltc management apr...              100   
73     wan na see social distance lend mf money lol ‍              100   
74                              new quarantine friend              100   

   final_keyword_match_Imp     user_id_y coordinates_y          location_y  \
0               quarantine  9.058951e+17           NaN                 NaN   
1               quarantine  3.391316e+07           NaN       New York, USA   
2                 lockdown  2.354005e+08           NaN          London, UK   
3          social distance  1.187517e+18           NaN           Vancouver   
4               quarantine  2.443667e+09           NaN          Boston, MA   
..                     ...           ...           ...                 ...   
70                lockdown  1.286625e+18           NaN                 NaN   
71                lockdown  1.923847e+07           NaN                  UK   
72                  isolat  1.233970e+18           NaN                 NaN   
73         social distance  1.177414e+18           NaN                 NaN   
74              quarantine  2.364339e+08           NaN  Clitheroe, England   

                 extended_tweet_full_text_duplicate_y  \
0   She isn’t in need, and she surely isn’t sugges...   
1   Mom surprises son with best friend reunion aft...   
2   12 weeks ago we had an idea to make a movie du...   
3   Don’t forget to practice social distancing y’a...   
4   Waiting for corona to end like...\n\n#jennifer...   
..                                                ...   
70  Please sign to end this lockdown  https://t.co...   
71  You know these adverts about people putting lo...   
72  @dg_grrl If getting help to mom depends on the...   
73  You wanna see social distancing?\nLend a mf so...   
74  @mindykaling New quarantine friend https://t.c...   

 

### 1_Implementation and 4_Social Disruption

In [93]:
pd.merge(interim_Imp, interim_Sd, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x                    location_x  \
0   1.283363e+18  3.068802e+08           NaN                           NaN   
1   1.294937e+18  4.265885e+09           NaN    Amsterdam, The Netherlands   
2   1.295439e+18  3.000085e+09           NaN                           NaN   
3   1.295458e+18  8.393254e+07           NaN                   Phoenix, AZ   
4   1.295620e+18  7.421430e+05           NaN                    London, UK   
5   1.295639e+18  4.427825e+08           NaN                         India   
6   1.295674e+18  1.241353e+18           NaN                            UK   
7   1.295684e+18  4.728404e+08           NaN                           MNL   
8   1.295752e+18  1.425158e+09           NaN                     Worldwide   
9   1.295758e+18  3.243421e+09           NaN                         India   
10  1.295759e+18  9.489464e+17           NaN               New Jersey, USA   
11  1.295768e+18  2.380828e+07           NaN                 Buffalo, N.Y.   
12  1.295769e+18  2.554900e+07           NaN                   Buffalo, NY   
13  1.295771e+18  3.835119e+07           NaN  I follow back patriots back    
14  1.295772e+18  2.529641e+09           NaN                   Dublin West   
15  1.295773e+18  4.987534e+08           NaN                      Cheshire   
16  1.295774e+18  7.282073e+07           NaN               Dublin, Ireland   
17  1.295781e+18  2.322682e+08           NaN                      New York   
18  1.295783e+18  1.212377e+18           NaN          Washington, DC/MD/VA   
19  1.295787e+18  1.557268e+07           NaN              Jacksonville, FL   
20  1.295788e+18  1.630997e+07           NaN                 Cleveland, OH   
21  1.295793e+18  8.070950e+05           NaN                 New York City   
22  1.295798e+18  1.256304e+18           NaN          Long Island New York   
23  1.295800e+18  2.197756e+09           NaN             Limerick, Ireland   
24  1.295802e+18  1.270036e+18           NaN            True Blue in Texas   
25  1.295803e+18  9.808502e+17           NaN                Twilight Zone    
26  1.295803e+18  2.449914e+09           NaN                  Last Outpost   
27  1.295804e+18  1.239944e+09           NaN                           NaN   
28  1.295805e+18  9.602853e+17           NaN                     Ohio, USA   
29  1.295805e+18  8.654976e+08           NaN                           NaN   
30  1.295805e+18  1.080406e+18           NaN               California, USA   
31  1.295805e+18  2.408386e+09           NaN           Taipei City, Taiwan   
32  1.295805e+18  1.269774e+18           NaN                        Oxford   
33  1.295805e+18  4.189032e+09           NaN                  Liberty City   
34  1.295805e+18  8.173860e+05           NaN                 United States   
35  1.295805e+18  1.057947e+18           NaN                     Dover, DE   
36  1.295805e+18  1.586445e+07           NaN                  New York, NY   
37  1.295806e+18  1.116776e+18           NaN                           NaN   
38  1.295806e+18  3.521293e+08           NaN              Murfreesboro, TN   
39  1.295806e+18  1.414230e+09           NaN                     Neverland   
40  1.295806e+18  8.221693e+17           NaN                            NY   
41  1.295806e+18  1.502230e+07           NaN                       vermont   
42  1.295806e+18  1.250537e+18           NaN                     Australia   
43  1.295806e+18  4.852020e+09           NaN                  Fredonia, NY   
44  1.295806e+18  2.517016e+09           NaN                     worldwide   
45  1.295806e+18  1.675406e+09           NaN         South Dublin, Ireland   
46  1.295806e+18  1.287258e+18           NaN                           NaN   
47  1.295806e+18  2.701849e+08           NaN                  Chicago area   
48  1.295806e+18  1.269774e+18           NaN                        Oxford   
49  1.295806e+18  1.288113e+18           NaN                           NaN   
50  1.295806e+18  1.007322e+08      

### 1_Implementation and 5_Purpose

In [94]:
pd.merge(interim_Imp, interim_Purp, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x                     location_x  \
0   1.292529e+18  5.076918e+07           NaN                             DC   
1   1.293971e+18  9.208697e+17           NaN                  United States   
2   1.295448e+18  1.176533e+18           NaN            Boiling Springs, NC   
3   1.295593e+18  4.922386e+07           NaN                  The High Seas   
4   1.295662e+18  1.704172e+07           NaN         Greater London, London   
5   1.295684e+18  4.728404e+08           NaN                            MNL   
6   1.295699e+18  1.072499e+18           NaN              Stamford, England   
7   1.295764e+18  2.045362e+07           NaN                 Sacramento, CA   
8   1.295772e+18  5.432296e+07           NaN                     Orange, CA   
9   1.295801e+18  1.276057e+08           NaN                 Washington, DC   
10  1.295802e+18  6.772865e+07           NaN                 North Carolina   
11  1.295805e+18  1.274763e+18           NaN                            NaN   
12  1.295805e+18  1.001892e+18           NaN                            NaN   
13  1.295805e+18  9.243243e+17           NaN                         Cyprus   
14  1.295805e+18  1.006789e+09           NaN  Plutocracy once known as USA.   
15  1.295806e+18  4.459331e+09           NaN                            NaN   
16  1.295806e+18  1.282685e+18           NaN                            NaN   
17  1.295806e+18  1.953147e+07           NaN                      edinburgh   
18  1.295806e+18  1.491978e+08           NaN                        Nigeria   
19  1.295806e+18  3.803548e+07           NaN                 Rio de Janeiro   
20  1.295806e+18  1.472824e+09           NaN                Baton Rouge, LA   
21  1.295806e+18  2.408765e+09           NaN            North West, England   
22  1.295806e+18  5.988054e+07           NaN                    Houston, TX   
23  1.295806e+18  9.476643e+17           NaN                   New York, NY   
24  1.295806e+18  4.820570e+08           NaN          Indianapolis, Indiana   

                 extended_tweet_full_text_duplicate_x  \
0   Conclusion: Lockdowns don’t work.  Sweden look...   
1   Thank you for the interview, @425Business! "Pr...   
2   Join us in caring for &amp; protecting our com...   
3   Spend all the money on the quarantine. Get peo...   
4   If you have coronavirus symptoms apply for a t...   
5                     This u? https://t.co/Z6tlaLCAYz   
6   @ActionBrexit With every passing day we see wi...   
7   Masks are one of the most effective ways to sl...   
8   The original rationale for severe social dista...   
9   The #SavethePostOffice fight is making a huge ...   
10  Quarantine sucks but we are now protected by G...   
11  Oh god ... Save us plzz ... \n@narendramodi #P...   
12  @marcdaalder I think we carve our own destiny....   
13  FINANCIAL ADVICE I WISH SOMEONE TOLD ME.... HO...   
14  @KathleenChu5 He thinks people will stay home ...   
15  @jrf195959 @ChantixMomFan @LtGovHochulNY @real...   
16  @realDonaldTrump Do executive order to give ho...   
17  @mr_eion But please don’t forget all the wonde...   
18                                      This is home!   
19  Coronavirus gets dangerously close to isolated...   
20  Wear your mask, social distance, stay away fro...   
21  Yet another "victory" for the lockdown acolyte...   
22  My favorite memory of BTS is when they legit s...   
23  @JumaaneWilliams @DNC Americans have to risk g...   
24  Working From Home Can Save You Money. What Sho...   

                                              final_x  final_score_Imp  \
0   conclusion lockdown  work sweden look to do co...              100   
1   thank the interview protect your asset role is...              100   
2   join u care amp protect community amp ourselve...              100   
3   spend the money the quarantine get people actu...              100   
4   coronavirus symptom apply test via call 119 se...              100   
5                               

### 1_Implementation and 6_Positive Emotion

In [95]:
pd.merge(interim_Imp, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x               location_x  \
0   1.295091e+18  1.237210e+18           NaN            East Bend, NC   
1   1.295382e+18  1.286631e+18           NaN                      NaN   
2   1.295456e+18  7.756275e+07           NaN          California, USA   
3   1.295519e+18  8.760420e+17           NaN             Florida Keys   
4   1.295587e+18  4.068424e+08           NaN                   London   
5   1.295598e+18  1.018661e+09           NaN   Gold Coast, Queensland   
6   1.295641e+18  1.906385e+08           NaN                      NaN   
7   1.295657e+18  2.009498e+07           NaN           London & Essex   
8   1.295662e+18  1.704172e+07           NaN   Greater London, London   
9   1.295717e+18  2.416792e+07           NaN                 Scotland   
10  1.295721e+18  5.456272e+08           NaN              The Bahamas   
11  1.295722e+18  7.402521e+17           NaN        Pennsylvania, USA   
12  1.295729e+18  7.464710e+17           NaN           The Bahamas 🇧🇸   
13  1.295754e+18  5.682496e+08           NaN           Worcestershire   
14  1.295758e+18  3.243421e+09           NaN                    India   
15  1.295758e+18  1.102702e+18           NaN             Portland, OR   
16  1.295771e+18  5.950400e+08           NaN                      NaN   
17  1.295774e+18  7.596274e+07           NaN        Jamshedpur, India   
18  1.295775e+18  5.077292e+07           NaN                Nashville   
19  1.295783e+18  2.650905e+07           NaN                 Scotland   
20  1.295792e+18  9.181539e+17           NaN    Jagatsinghapur, India   
21  1.295796e+18  6.440105e+07           NaN           Harrisburg, PA   
22  1.295797e+18  1.083121e+18           NaN        Nantwich, England   
23  1.295797e+18  1.233992e+18           NaN           Haryana, India   
24  1.295798e+18  3.781504e+07           NaN         Tennis Ball Fuzz   
25  1.295802e+18  1.269919e+18           NaN                    India   
26  1.295802e+18  1.294334e+18           NaN                      NaN   
27  1.295803e+18  1.598695e+07           NaN           Sacramento, CA   
28  1.295804e+18  1.969755e+08           NaN                     Cork   
29  1.295805e+18  1.405530e+07           NaN        Fairfield, CT USA   
30  1.295805e+18  8.553028e+08           NaN      Benin City, Nigeria   
31  1.295805e+18  4.160250e+08           NaN                      NaN   
32  1.295805e+18  1.268273e+18           NaN                      NaN   
33  1.295805e+18  5.977951e+07           NaN     Hyderabad . Toronto    
34  1.295805e+18  1.967207e+07           NaN            Dallas, Texas   
35  1.295805e+18  1.145759e+18           NaN                      NaN   
36  1.295805e+18  3.169509e+08           NaN                    Lagos   
37  1.295805e+18  1.295263e+18           NaN                      NaN   
38  1.295805e+18  2.523387e+07           NaN          Palmdale, Calif   
39  1.295806e+18  3.338142e+08           NaN                  Arizona   
40  1.295806e+18  1.295379e+18           NaN                      NaN   
41  1.295806e+18  2.517016e+09           NaN                worldwide   
42  1.295806e+18  1.295311e+18           NaN                      NaN   
43  1.295806e+18  3.389387e+09           NaN                  Georgia   
44  1.295806e+18  4.861810e+09           NaN                      NaN   
45  1.295806e+18  2.208640e+07           NaN               California   
46  1.295806e+18  8.872712e+06           NaN  Silicon Valley, CA, USA   
47  1.295806e+18  4.500585e+07           NaN             San Jose, CA   
48  1.295806e+18  1.220421e+18           NaN                      NaN   

                 extended_tweet_full_text_duplicate_x  \
0   In NC schools desks are “safely distanced”. An...   
1   @PMOIndia @DrRPNishank @DG_NTA @HRDMinistry  #...   
2   This is my daughters Skye’s 1st day of school ...   
3   WE'RE ALL EXPERIENCING CHALLENGING TIMES RIGHT...   
4   Yikes! No desire to dump on those starting out...   
5   @hughriminton You

### 2_Adaptation and 3_Negative Emotion

In [96]:
pd.merge(interim_Ada, interim_Ne, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x       location_x  \
0   1.295456e+18  7.756275e+07           NaN  California, USA   
1   1.295684e+18  4.728404e+08           NaN              MNL   
2   1.295693e+18  1.446385e+07           NaN   Washington, DC   
3   1.295805e+18  6.176812e+07           NaN    London, UK 🇬🇧   
4   1.295805e+18  1.022517e+08           NaN              NYC   
5   1.295805e+18  7.258755e+17           NaN     Florida, USA   
6   1.295805e+18  1.268295e+18           NaN         Kekistan   
7   1.295805e+18  2.781876e+08           NaN   Alexandria, VA   
8   1.295806e+18  1.240045e+18           NaN              NaN   
9   1.295806e+18  6.565398e+07           NaN    West Bend, WI   
10  1.295806e+18  1.557711e+07           NaN      New Zealand   
11  1.295806e+18  1.267086e+18           NaN            Texas   

                 extended_tweet_full_text_duplicate_x  \
0   This is my daughters Skye’s 1st day of school ...   
1                     This u? https://t.co/Z6tlaLCAYz   
2   In just months, #COVID19 has set back a decade...   
3   When I tell you I YELL!!! THEY CHANGED THE HYP...   
4   Excited to go on my friend @AymanM’s show in t...   
5   And further remember that Manafort was sentenc...   
6   @SpeakerPelosi Nancy wanted to go on a 3-week ...   
7   — Independent Agencies Report Lost Revenue, Cl...   
8   @NYGovCuomo you killed my family and friends n...   
9   Summer fun! The majority of our associates are...   
10  Coronavirus: Musician still in NZ after five m...   
11  Went to @cvspharmacy today to pickup Rx for a ...   

                                              final_x  final_score_Ada  \
0   daughter skye  1st day school pictureshe  actu...              100   
1                                                   u              100   
2   month covid19 set back decade progress the bat...              100   
3   tell yell change the hyperlink to go to game t...              100   
4   excite to go friend  show the 3 pm hour tune h...              100   
5   remember manafort sentence to 7 12 year federa...              100   
6   nancy want to go 3week vacation pandemic amp h...              100   
7   independent agency report lose revenue client ...              100   
8   kill family friend support u give u aid fuck p...              100   
9   summer fun the majority associate still work f...              100   
10  coronavirus musician still nz five month scary...              100   
11  go to today to pickup rx friend try to pickup ...              100   

   final_keyword_match_Ada     user_id_y coordinates_y       location_y  \
0                    learn  7.756275e+07           NaN  California, USA   
1          online purchase  4.728404e+08           NaN              MNL   
2                    learn  1.446385e+07           NaN   Washington, DC   
3                    to go  6.176812e+07           NaN    London, UK 🇬🇧   
4                    to go  1.022517e+08           NaN              NYC   
5                    to go  7.258755e+17           NaN     Florida, USA   
6                    to go  1.268295e+18           NaN         Kekistan   
7                      wfh  2.781876e+08           NaN   Alexandria, VA   
8                    to go  1.240045e+18           NaN              NaN   
9           work from home  6.565398e+07           NaN    West Bend, WI   
10                   to go  1.557711e+07           NaN      New Zealand   
11              drive thru  1.267086e+18           NaN            Texas   

                 extended_tweet_full_text_duplicate_y  \
0   This is my daughters Skye’s 1st day of school ...   
1                     This u? https://t.co/Z6tlaLCAYz   
2   In just months, #COVID19 has set back a decade...   
3   When I tell you I YELL!!! THEY CHANGED THE HYP...   
4   Excited to go on my friend @AymanM’s show in t...   
5   And further remember that Manafort was sentenc...   
6   @SpeakerPelosi Nancy wanted to go on a 3-week ...   
7   — Independent Agencies Repo

### 2_Adaptation and 4_Social Disruption

In [97]:
pd.merge(interim_Ada, interim_Sd, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x        location_x  \
0  1.295684e+18  4.728404e+08           NaN               MNL   
1  1.295704e+18  3.343238e+09           NaN     Kumasi, Ghana   
2  1.295774e+18  7.282073e+07           NaN   Dublin, Ireland   
3  1.295804e+18  1.239944e+09           NaN               NaN   
4  1.295805e+18  1.271343e+18           NaN      South Africa   
5  1.295806e+18  3.521293e+08           NaN  Murfreesboro, TN   
6  1.295806e+18  6.106590e+08           NaN            Oxford   
7  1.295806e+18  1.901592e+09           NaN   Los Angeles, CA   

                extended_tweet_full_text_duplicate_x  \
0                    This u? https://t.co/Z6tlaLCAYz   
1  @jeffwellz Sometimes i wish that those who arg...   
2  Ireland’s new #COVID19 measures⤵️\n\n💻Work fro...   
3  This is going to be an unpopular opinion but I...   
4  Melanin T-shirts Available!\n\nSize : Large (N...   
5  One of the best parts about working from home ...   
6  New in the Journal of Public Economics.\n\nBas...   
7  Read our latest blog article to learn what to ...   

                                             final_x  final_score_Ada  \
0                                                  u              100   
1  sometimes wish argue unnecessary precautionary...              100   
2  ireland  new covid19 measure work from home po...              100   
3  go to unpopular opinion think need to go full ...              100   
4  melanin tshirts available size large nb tshirt...              100   
5  one the best part work from home amp  always p...              100   
6  new the journal public economics base real tim...              100   
7  read late blog article to learn to expect fly ...              100   

  final_keyword_match_Ada     user_id_y coordinates_y        location_y  \
0         online purchase  4.728404e+08           NaN               MNL   
1                   learn  3.343238e+09           NaN     Kumasi, Ghana   
2          work from home  7.282073e+07           NaN   Dublin, Ireland   
3                   to go  1.239944e+09           NaN               NaN   
4                delivery  1.271343e+18           NaN      South Africa   
5          work from home  3.521293e+08           NaN  Murfreesboro, TN   
6          work from home  6.106590e+08           NaN            Oxford   
7                   learn  1.901592e+09           NaN   Los Angeles, CA   

                extended_tweet_full_text_duplicate_y  \
0                    This u? https://t.co/Z6tlaLCAYz   
1  @jeffwellz Sometimes i wish that those who arg...   
2  Ireland’s new #COVID19 measures⤵️\n\n💻Work fro...   
3  This is going to be an unpopular opinion but I...   
4  Melanin T-shirts Available!\n\nSize : Large (N...   
5  One of the best parts about working from home ...   
6  New in the Journal of Public Economics.\n\nBas...   
7  Read our latest blog article to learn what to ...   

                                             final_y  final_score_Sd  \
0                                                  u             100   
1  sometimes wish argue unnecessary precautionary...             100   
2  ireland  new covid19 measure work from home po...             100   
3  go to unpopular opinion think need to go full ...             100   
4  melanin tshirts available size large nb tshirt...             100   
5  one the best part work from home amp  always p...             100   
6  new the journal public economics base real tim...             100   
7  read late blog article to learn to expect fly ...             100   

  final_keyword_match_Sd  
0        social function  
1            unnecessary  
2              gathering  
3                 travel  
4                  large  
5            interaction  
6                  large  
7                 travel

### 2_Adaptation and 5_Purpose

In [98]:
pd.merge(interim_Ada, interim_Purp, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x                      location_x  \
0   1.292883e+18  1.917750e+07           NaN             Geneva, Switzerland   
1   1.295684e+18  4.728404e+08           NaN                             MNL   
2   1.295728e+18  5.493593e+07           NaN                        Room 101   
3   1.295768e+18  3.238207e+07           NaN                             NaN   
4   1.295798e+18  2.256296e+07           NaN                         Toronto   
5   1.295805e+18  1.290319e+18           NaN                             NaN   
6   1.295805e+18  9.719671e+17           NaN                   In your veins   
7   1.295805e+18  2.352289e+09           NaN               Silver Spring, MD   
8   1.295805e+18  1.425606e+09           NaN             North Carolina, USA   
9   1.295805e+18  9.243243e+17           NaN                          Cyprus   
10  1.295806e+18  1.363893e+08           NaN  Virginia Beach, Virginia 23451   
11  1.295806e+18  1.491978e+08           NaN                         Nigeria   
12  1.295806e+18  8.439271e+07           NaN                             NaN   
13  1.295806e+18  4.820570e+08           NaN           Indianapolis, Indiana   

                 extended_tweet_full_text_duplicate_x  \
0   The lessons learned in fighting HIV can be use...   
1                     This u? https://t.co/Z6tlaLCAYz   
2   Well, yes. As we said they would.\n\nNow that ...   
3   Webinar | Canada | With #COVID19, there has be...   
4   Your mask protects others &amp; their mask pro...   
5   How data visualization saved 40,000+ lives thi...   
6   I did learn one thing Bc of COVID. I don’t nee...   
7   Has your child missed a vaccine appointment? N...   
8   Following the Three Ws is a great way to #Stop...   
9   FINANCIAL ADVICE I WISH SOMEONE TOLD ME.... HO...   
10  Sure would be good if Virginia Executive and L...   
11                                      This is home!   
12  Lol- sometimes you have to shake your head! Di...   
13  Working From Home Can Save You Money. What Sho...   

                                              final_x  final_score_Ada  \
0   the lesson learn fight hiv use the response to...              100   
1                                                   u              100   
2   well yes say would mask  ‘ give confidence to ...              100   
3   webinar canada covid19 alarm rise domesticviol...              100   
4   your mask protects others amp mask protects we...              100   
5   data visualization save 40000 life spring lear...              100   
6   learn one thing bc covid  need the nail salon ...              100   
7   your child miss vaccine appointment the time t...              100   
8   follow the three w great way to stopthespread ...              100   
9   financial advice wish someone told to save mon...              100   
10  sure would good virginia executive legislator ...              100   
11                                               home              100   
12  lol sometimes to shake your head occur to god ...              100   
13                       work from home save money do              100   

   final_keyword_match_Ada     user_id_y coordinates_y  \
0                    learn  1.917750e+07           NaN   
1          online purchase  4.728404e+08           NaN   
2                    to go  5.493593e+07           NaN   
3                 telework  3.238207e+07           NaN   
4                    learn  2.256296e+07           NaN   
5                    learn  1.290319e+18           NaN   
6                    learn  9.719671e+17           NaN   
7                    learn  2.352289e+09           NaN   
8                    learn  1.425606e+09           NaN   
9           work from home  9.243243e+17           NaN   
10                   learn  1.363893e+08           NaN   
11        school from home  1.491978e+08           NaN   
12                   to go  8.439271e+07           NaN   
13          work from home  4.8205

### 2_Adaptation and 6_Positive Emotion

In [99]:
pd.merge(interim_Ada, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x             location_x  \
0   1.294969e+18  1.262649e+18           NaN                    NaN   
1   1.295383e+18  2.479775e+08           NaN  Everywhere I tweet...   
2   1.295456e+18  7.756275e+07           NaN        California, USA   
3   1.295801e+18  1.171791e+18           NaN         Washington, DC   
4   1.295805e+18  1.019949e+18           NaN         Birmingham, AL   
5   1.295805e+18  1.406983e+07           NaN           Thornton, CO   
6   1.295805e+18  1.360542e+09           NaN                    NaN   
7   1.295806e+18  9.726738e+08           NaN                    NaN   
8   1.295806e+18  1.349508e+08           NaN                    NaN   
9   1.295806e+18  9.032070e+08           NaN               New York   
10  1.295806e+18  1.901592e+09           NaN        Los Angeles, CA   

                 extended_tweet_full_text_duplicate_x  \
0   There's just four weeks to go until the Taunto...   
1   Our enemies tells Us that We have to wear a ma...   
2   This is my daughters Skye’s 1st day of school ...   
3   As we continue to practice #SocialDistancing, ...   
4   Don't let the heat sneak up on you this #Toolb...   
5   The workbook for the award-winning Virtual Div...   
6   School districts: "School is now remote in fal...   
7   Today’s cloud computing solutions keep small b...   
8   I genuinely would rather go back to the office...   
9   @MaggieCaroline1 My mother once needed a rabie...   
10  Read our latest blog article to learn what to ...   

                                              final_x  final_score_Ada  \
0   there four week to go the taunton independent ...              100   
1   enemy tell u to wear mask to go anywhere to sh...              100   
2   daughter skye  1st day school pictureshe  actu...              100   
3   continue to practice socialdistancing teleheal...              100   
4   dont let the heat sneak toolboxtalktuesday lea...              100   
5   the workbook the awardwinning virtual divide a...              100   
6   school district school remote fall safe need t...              100   
7   today  cloud compute solution keep small busin...              100   
8   genuinely would rather go back to the office g...              100   
9   mother need rabies vaccine infrequently give a...              100   
10  read late blog article to learn to expect fly ...              100   

   final_keyword_match_Ada     user_id_y coordinates_y             location_y  \
0                    to go  1.262649e+18           NaN                    NaN   
1                    to go  2.479775e+08           NaN  Everywhere I tweet...   
2                    learn  7.756275e+07           NaN        California, USA   
3               telehealth  1.171791e+18           NaN         Washington, DC   
4                    learn  1.019949e+18           NaN         Birmingham, AL   
5                   remote  1.406983e+07           NaN           Thornton, CO   
6                   remote  1.360542e+09           NaN                    NaN   
7                   remote  9.726738e+08           NaN                    NaN   
8                     zoom  1.349508e+08           NaN                    NaN   
9                    to go  9.032070e+08           NaN               New York   
10                   learn  1.901592e+09           NaN        Los Angeles, CA   

                 extended_tweet_full_text_duplicate_y  \
0   There's just four weeks to go until the Taunto...   
1   Our enemies tells Us that We have to wear a ma...   
2   This is my daughters Skye’s 1st day of school ...   
3   As we continue to practice #SocialDistancing, ...   
4   Don't let the heat sneak up on you this #Toolb...   
5   The workbook for the award-winning Virtual Div...   
6   School districts: "School is now remote in fal...   
7   Today’s cloud computing solutions keep small b...   
8   I genuinely would rather go back to the office...   
9   @MaggieCaroline1 My mother once needed a

### 3_Negative Emotion and 4_Social Disruption

In [100]:
pd.merge(interim_Ne, interim_Sd, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x         location_x  \
0   1.294368e+18  1.580378e+09           NaN       Hartford, CT   
1   1.294754e+18  7.222420e+08           NaN                NaN   
2   1.295346e+18  3.806486e+08           NaN             France   
3   1.295557e+18  3.806486e+08           NaN             France   
4   1.295684e+18  4.728404e+08           NaN                MNL   
5   1.295709e+18  3.347158e+08           NaN      Manhattan, NY   
6   1.295768e+18  1.083143e+09           NaN      Cork, Ireland   
7   1.295805e+18  1.168542e+08           NaN      United States   
8   1.295806e+18  1.502230e+07           NaN            vermont   
9   1.295806e+18  2.701849e+08           NaN       Chicago area   
10  1.295806e+18  1.328691e+08           NaN        Seattle, WA   
11  1.295806e+18  1.765451e+07           NaN  Bar Harbor, Maine   

                 extended_tweet_full_text_duplicate_x  \
0   The town of Mansfield worries UConn's reopenin...   
1   They’re doing that thing again. Announcing maj...   
2   VIDEO: 🇨🇳 Crowds packed out a water park over ...   
3   VIDEO:  🇨🇳Crowds packed out a water park over ...   
4                     This u? https://t.co/Z6tlaLCAYz   
5   Fauci has no real answers or insights, his onl...   
6   This bothers me no end. The virus doesn't give...   
7   While #gapyears have become an increasingly po...   
8   @chopaganda listen, friend, your risks are you...   
9   Yesterday I wondered how and why I ended up at...   
10  If you are bummed about your travel plans bein...   
11  We may be at the end of summer, but mask-weari...   

                                              final_x  final_score_Ne  \
0   the town mansfield worry uconns reopen could l...             100   
1    thing announce major policy decision sat nigh...             100   
2   video crowd packed out water park the weekend ...             100   
3   video crowd packed out water park the weekend ...             100   
4                                                   u             100   
5   fauci no real answer insight prevention advice...             100   
6   bother no end the virus doesnt give toss the c...             100   
7   gapyears become increasingly popular trend the...             100   
8   listen friend your risk to make risk to others...             100   
9   yesterday wonder end at today realize home ald...             100   
10  bum your travel plan disrupt due to the global...             100   
11  may at the end summer maskwearing sanitizertot...             100   

         final_keyword_match_Ne     user_id_y coordinates_y  \
0                         worry  1.580378e+09           NaN   
1                           end  7.222420e+08           NaN   
2                           end  3.806486e+08           NaN   
3                           end  3.806486e+08           NaN   
4   sideeffectsofquarantinelife  4.728404e+08           NaN   
5                       anxiety  3.347158e+08           NaN   
6                           end  1.083143e+09           NaN   
7                           end  1.168542e+08           NaN   
8                           end  1.502230e+07           NaN   
9                       anxiety  2.701849e+08           NaN   
10                        worry  1.328691e+08           NaN   
11                          end  1.765451e+07           NaN   

           location_y               extended_tweet_full_text_duplicate_y  \
0        Hartford, CT  The town of Mansfield worries UConn's reopenin...   
1                 NaN  They’re doing that thing again. Announcing maj...   
2              France  VIDEO: 🇨🇳 Crowds packed out a water park over ...   
3              France  VIDEO:  🇨🇳Crowds packed out a water park over ...   
4                 MNL                    This u? https://t.co/Z6tlaLCAYz   
5       Manhattan, NY  Fauci has no real answers or insights, his onl...   
6       Cork, Ireland  This bothers me no end. The virus doesn't give...   
7       United States 

### 3_Negative Emotion and 5_Purpose

In [101]:
pd.merge(interim_Ne, interim_Purp, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x                   location_x  \
0  1.295593e+18  4.922386e+07           NaN                The High Seas   
1  1.295684e+18  4.728404e+08           NaN                          MNL   
2  1.295805e+18  1.226956e+18           NaN                Palo Alto, CA   
3  1.295805e+18  7.814128e+17           NaN  वैशाली, मथुरा, अंडमान, भारत   
4  1.295806e+18  3.403295e+09           NaN          Manchester, England   
5  1.295806e+18  1.908231e+07           NaN                    Baltimore   

                extended_tweet_full_text_duplicate_x  \
0  Spend all the money on the quarantine. Get peo...   
1                    This u? https://t.co/Z6tlaLCAYz   
2  "The moratorium extends protections toward ten...   
3  Let's follow up...\nTest\nAllow Testing atleas...   
4                       saved 2020 ended coronavirus   
5  Catholic sociologist and gender expert, Miriam...   

                                             final_x  final_score_Ne  \
0  spend the money the quarantine get people actu...             100   
1                                                  u             100   
2  the moratorium extends protection toward tenan...             100   
3  let follow test allow test atleast infact the ...             100   
4                          save 2020 end coronavirus             100   
5  catholic sociologist gender expert miriam rahi...             100   

        final_keyword_match_Ne     user_id_y coordinates_y  \
0                        worry  4.922386e+07           NaN   
1  sideeffectsofquarantinelife  4.728404e+08           NaN   
2                          end  1.226956e+18           NaN   
3                          end  7.814128e+17           NaN   
4                          end  3.403295e+09           NaN   
5                          end  1.908231e+07           NaN   

                    location_y  \
0                The High Seas   
1                          MNL   
2                Palo Alto, CA   
3  वैशाली, मथुरा, अंडमान, भारत   
4          Manchester, England   
5                    Baltimore   

                extended_tweet_full_text_duplicate_y  \
0  Spend all the money on the quarantine. Get peo...   
1                    This u? https://t.co/Z6tlaLCAYz   
2  "The moratorium extends protections toward ten...   
3  Let's follow up...\nTest\nAllow Testing atleas...   
4                       saved 2020 ended coronavirus   
5  Catholic sociologist and gender expert, Miriam...   

                                             final_y  final_score_Purp  \
0  spend the money the quarantine get people actu...               100   
1                                                  u               100   
2  the moratorium extends protection toward tenan...               100   
3  let follow test allow test atleast infact the ...               100   
4                          save 2020 end coronavirus               100   
5  catholic sociologist gender expert miriam rahi...               100   

  final_keyword_match_Purp  
0                     save  
1        flatten the curve  
2                  protect  
3                     save  
4                     save  
5                     save

### 3_Negative Emotion and 6_Positive Emotion

In [102]:
pd.merge(interim_Ne, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x             location_x  \
0   1.290330e+18  2.987672e+09           NaN          Annapolis, MD   
1   1.295456e+18  7.756275e+07           NaN        California, USA   
2   1.295519e+18  8.760420e+17           NaN           Florida Keys   
3   1.295676e+18  3.571448e+08           NaN                    NaN   
4   1.295738e+18  1.124611e+08           NaN  Dumfries and Galloway   
5   1.295746e+18  2.399638e+08           NaN                    NaN   
6   1.295758e+18  1.102702e+18           NaN           Portland, OR   
7   1.295763e+18  2.240798e+09           NaN                Earth X   
8   1.295776e+18  1.160164e+08           NaN                    NaN   
9   1.295805e+18  1.208852e+18           NaN     Dehli Pakistan 🇵🇰    
10  1.295805e+18  1.967207e+07           NaN          Dallas, Texas   
11  1.295806e+18  2.646467e+09           NaN             Cyberspace   
12  1.295806e+18  1.765451e+07           NaN      Bar Harbor, Maine   
13  1.295806e+18  6.690661e+07           NaN      Westerville, Ohio   

                 extended_tweet_full_text_duplicate_x  \
0   I have issued an amended emergency order ensur...   
1   This is my daughters Skye’s 1st day of school ...   
2   WE'RE ALL EXPERIENCING CHALLENGING TIMES RIGHT...   
3   Good day my friends \m/ Positive mind &amp; Ke...   
4   We're seeing a few face masks being discarded ...   
5   GET INTO IT!!! Click the link if you’re lookin...   
6   Congratulations to our friends at @transitionp...   
7   God will punish whoever started this fake tren...   
8   Derek Carr said he feels for Marcus Mariota in...   
9   59k plus corona virus cases today in endia Bak...   
10  We were so excited to host an event over the w...   
11  @originalalison @Tauranga2020 @vincristine @ca...   
12  We may be at the end of summer, but mask-weari...   
13  @GovMikeDeWine I understand however, I worry a...   

                                              final_x  final_score_Ne  \
0   issue amend emergency order ensure local schoo...             100   
1   daughter skye  1st day school pictureshe  actu...             100   
2   experience challenge time right hopeful call f...             100   
3   good day friend positive mind amp keep brutal ...             100   
4   see face mask discard around town centre pleas...             100   
5   get click the link  look covid safe event labo...             100   
6   congratulation to friend at prioritize safe pl...             100   
7   god punish whoever start fake trend give u fal...             100   
8   derek carr say feel marcus mariota new offense...             100   
9   59k plus corona virus case today endia bakshi ...             100   
10  excite to host event the weekend complete soci...             100   
11  medias agenda always to take jacinda labour to...             100   
12  may at the end summer maskwearing sanitizertot...             100   
13  understand however worry aggressive coach fixa...             100   

   final_keyword_match_Ne     user_id_y coordinates_y             location_y  \
0                     end  2.987672e+09           NaN          Annapolis, MD   
1                     end  7.756275e+07           NaN        California, USA   
2                     end  8.760420e+17           NaN           Florida Keys   
3                     end  3.571448e+08           NaN                    NaN   
4                    scar  1.124611e+08           NaN  Dumfries and Galloway   
5                     end  2.399638e+08           NaN                    NaN   
6                     end  1.102702e+18           NaN           Portland, OR   
7                     end  2.240798e+09           NaN                Earth X   
8                   worry  1.160164e+08           NaN                    NaN   
9                     end  1.208852e+18           NaN     Dehli Pakistan 🇵🇰    
10                    end  1.967207e+07           NaN          Dallas, Texas   
11                    end  2.646467e

### 4_Social Disruption and 5_Purpose

In [103]:
pd.merge(interim_Sd, interim_Purp, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x       location_x  \
0  1.295684e+18  4.728404e+08           NaN              MNL   
1  1.295694e+18  8.166339e+17           NaN    United States   
2  1.295806e+18  1.472824e+09           NaN  Baton Rouge, LA   

                extended_tweet_full_text_duplicate_x  \
0                    This u? https://t.co/Z6tlaLCAYz   
1  Let me break this down for you Democrats. You ...   
2  Wear your mask, social distance, stay away fro...   

                                             final_x  final_score_Sd  \
0                                                  u             100   
1  let break democrat save auto worker job trump ...             100   
2  wear your mask social distance stay away from ...             100   

  final_keyword_match_Sd     user_id_y coordinates_y       location_y  \
0        social function  4.728404e+08           NaN              MNL   
1                 travel  8.166339e+17           NaN    United States   
2                  large  1.472824e+09           NaN  Baton Rouge, LA   

                extended_tweet_full_text_duplicate_y  \
0                    This u? https://t.co/Z6tlaLCAYz   
1  Let me break this down for you Democrats. You ...   
2  Wear your mask, social distance, stay away fro...   

                                             final_y  final_score_Purp  \
0                                                  u               100   
1  let break democrat save auto worker job trump ...               100   
2  wear your mask social distance stay away from ...               100   

  final_keyword_match_Purp  
0        flatten the curve  
1                     save  
2                  protect

### 4_Social Disruption and 6_Positive Emotion

In [104]:
pd.merge(interim_Sd, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x                location_x  \
0   1.295509e+18  2.044551e+07           NaN           Dublin, Ireland   
1   1.295722e+18  1.022307e+07           NaN      Philadelphia, PA USA   
2   1.295748e+18  8.637246e+17           NaN  East Riding Of Yorkshire   
3   1.295758e+18  3.243421e+09           NaN                     India   
4   1.295765e+18  4.573406e+09           NaN             Mumbai, India   
5   1.295805e+18  1.174285e+18           NaN                       NaN   
6   1.295805e+18  2.079505e+08           NaN      Jerusalem, Palestine   
7   1.295806e+18  2.517016e+09           NaN                 worldwide   
8   1.295806e+18  1.765451e+07           NaN         Bar Harbor, Maine   
9   1.295806e+18  1.901592e+09           NaN           Los Angeles, CA   
10  1.295806e+18  2.410821e+09           NaN                 Singapore   

                 extended_tweet_full_text_duplicate_x  \
0   Sharing this again in the (fading) hope that s...   
1   ✅ Avoid large gatherings.\n✅ Maintain 6 feet b...   
2   Please find Covid-19 stats for 18th Aug. Sorry...   
3   I have 0 hope from u guys! Would just say plz ...   
4   Strict legal action will be taken and vehicles...   
5                                        Safe travels   
6   "A large number of people" running Direct Prov...   
7   Coronavirus Update: New Jersey adds Delaware b...   
8   We may be at the end of summer, but mask-weari...   
9   Read our latest blog article to learn what to ...   
10  Coronavirus and Travel: How to Travel Safe Dur...   

                                              final_x  final_score_Sd  \
0   share the fading hope someone might actually a...             100   
1   avoid large gathering maintain 6 foot others i...             100   
2   please find covid19 stats 18th aug sorry miss ...             100   
3   0 hope from u guy would say plz respect the ac...             100   
4   strict legal action take vehicle seize nonperm...             100   
5                                         safe travel             100   
6   large number people run direct provision centr...             100   
7   coronavirus update new jersey add delaware bac...             100   
8   may at the end summer maskwearing sanitizertot...             100   
9   read late blog article to learn to expect fly ...             100   
10  coronavirus travel to travel safe pandemic tim...             100   

   final_keyword_match_Sd     user_id_y coordinates_y  \
0                   crowd  2.044551e+07           NaN   
1               gathering  1.022307e+07           NaN   
2                  travel  8.637246e+17           NaN   
3                   crowd  3.243421e+09           NaN   
4            nonessential  4.573406e+09           NaN   
5                  travel  1.174285e+18           NaN   
6                   large  2.079505e+08           NaN   
7                  travel  2.517016e+09           NaN   
8                  travel  1.765451e+07           NaN   
9                  travel  1.901592e+09           NaN   
10                 travel  2.410821e+09           NaN   

                  location_y  \
0            Dublin, Ireland   
1       Philadelphia, PA USA   
2   East Riding Of Yorkshire   
3                      India   
4              Mumbai, India   
5                        NaN   
6       Jerusalem, Palestine   
7                  worldwide   
8          Bar Harbor, Maine   
9            Los Angeles, CA   
10                 Singapore   

                 extended_tweet_full_text_duplicate_y  \
0   Sharing this again in the (fading) hope that s...   
1   ✅ Avoid large gatherings.\n✅ Maintain 6 feet b...   
2   Please find Covid-19 stats for 18th Aug. Sorry...   
3   I have 0 hope from u guys! Would just say plz ...   
4   Strict legal action will be taken and vehicles...   
5                                        Safe travels   
6   "A large number of people" running Direct Prov...   
7   Coronavirus Update: New Jersey ad

### 5_Purpose and 6_Positive Emotion

In [105]:
pd.merge(interim_Purp, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

id     user_id_x coordinates_x               location_x  \
0   1.295390e+18  3.703170e+08           NaN         Toronto, Ontario   
1   1.295662e+18  1.704172e+07           NaN   Greater London, London   
2   1.295707e+18  8.204525e+17           NaN             New York, NY   
3   1.295740e+18  2.479690e+08           NaN           Portsmouth, UK   
4   1.295746e+18  3.013805e+08           NaN              Bhubaneswar   
5   1.295755e+18  9.195554e+17           NaN                   Canada   
6   1.295803e+18  3.392064e+08           NaN  Los Angeles, California   
7   1.295805e+18  1.212085e+18           NaN                      NaN   
8   1.295805e+18  2.261457e+07           NaN           Greensboro, NC   
9   1.295806e+18  8.207278e+17           NaN                Miami, FL   
10  1.295806e+18  3.359287e+07           NaN           Burlington, VT   

                 extended_tweet_full_text_duplicate_x  \
0   Investing in school safety will save more live...   
1   If you have coronavirus symptoms apply for a t...   
2   Am I hopeful we will have a vaccine in the com...   
3   The COVID19 crisis - only science will save us...   
4   Also provide her Personal Protective equipment...   
5   This is why it’s so important we re-open schoo...   
6   BREAKING: After a COVID outbreak at the store,...   
7   #Checkout this #new #facemask #fineartamerica ...   
8   It's as important as ever to stay safe and pro...   
9   American Spa Magazine has published CE Institu...   
10  While I wish that this semester was not bringi...   

                                              final_x  final_score_Purp  \
0   invest school safety save life buying 40000 ve...               100   
1   coronavirus symptom apply test via call 119 se...               100   
2   hopeful vaccine the come month yes do think ge...               100   
3   the covid19 crisis science save u right scienc...               100   
4   also provide personal protective equipment ppe...               100   
5    important reopen school safely spend the mone...               100   
6   break covid outbreak at the store worker at 36...               100   
7   checkout new facemask fineartamerica covid19 p...               100   
8   important ever to stay safe protect from covid...               100   
9   american spa magazine publish ce institute dat...               100   
10  wish semester bring student back to town since...               100   

   final_keyword_match_Purp     user_id_y coordinates_y  \
0                      save  3.703170e+08           NaN   
1                   protect  1.704172e+07           NaN   
2                   protect  8.204525e+17           NaN   
3                      save  2.479690e+08           NaN   
4                   protect  3.013805e+08           NaN   
5                      save  9.195554e+17           NaN   
6                   protect  3.392064e+08           NaN   
7                   protect  1.212085e+18           NaN   
8                   protect  2.261457e+07           NaN   
9                   protect  8.207278e+17           NaN   
10                  protect  3.359287e+07           NaN   

                 location_y  \
0          Toronto, Ontario   
1    Greater London, London   
2              New York, NY   
3            Portsmouth, UK   
4               Bhubaneswar   
5                    Canada   
6   Los Angeles, California   
7                       NaN   
8            Greensboro, NC   
9                 Miami, FL   
10           Burlington, VT   

                 extended_tweet_full_text_duplicate_y  \
0   Investing in school safety will save more live...   
1   If you have coronavirus symptoms apply for a t...   
2   Am I hopeful we will have a vaccine in the com...   
3   The COVID19 crisis - only science will save us...   
4   Also provide her Personal Protective equipment...   
5   This is why it’s so important we re-open schoo...   
6   BREAKING: After a COVID outbreak at the store,...   
7   #Checkout